# AI 로그 분석 시스템 - LLM 체인 테스트

**목적**: LangChain 체인이 다양한 입력에 대해 올바르게 동작하는지 테스트

**요구사항**:
- OPENAI_API_KEY 환경변수 설정
- OpenSearch, Kafka 불필요

**테스트 항목**:
1. 로그 분석 체인 (log_analysis_chain)
2. 챗봇 RAG 체인 (chatbot_chain)

## 1. 환경 설정

In [31]:
%pip install langchain-openai matplotlib
%pip install python-dotenv langchain langchain-openai langchain-community langchain-text-splitters docx2txt langchain-chroma
%pip install pandas

     ---------------------------------------- 0.0/114.6 kB ? eta -:--:--
     -------------- ------------------------- 41.0/114.6 kB ? eta -:--:--
     -------------------------------------- 114.6/114.6 kB 6.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.1 MB 30.1 MB/s eta 0:00:01
   --- ------------------------------------ 0.8/8.1 MB 12.2 MB/s eta 0:00:01
   ------ --------------------------------- 1.2/8.1 MB 15.4 MB/s eta 0:00:01
   ------- -------------------------------- 1.5/8.1 MB 12.1 MB/s eta 0:00:01
   ----------- ---------------------------- 2.3/8.1 MB 13.0 MB/s eta 0:00:01
   ------------- -------------------------- 2.8/8.1 MB 13.8 MB/s eta 0:00:01
   ----------------- ---------------------- 3.5/8.1 MB 14.9 MB/s eta 0:00:01
   ------------------- -------------------- 3.9/8.1 MB 14.5 MB/s eta 0:00:01
   ---------------------- ----------------- 4.5/8.1 MB 14.4 MB/s eta 0:00:01
   --------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: c:\Users\God\.pyenv\pyenv-win\versions\3.11.9\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: c:\Users\God\.pyenv\pyenv-win\versions\3.11.9\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: c:\Users\God\.pyenv\pyenv-win\versions\3.11.9\python.exe -m pip install --upgrade pip


In [32]:
import sys
import os
from datetime import datetime
from pprint import pprint
import time

# 프로젝트 루트를 Python path에 추가
sys.path.insert(0, os.path.dirname(os.path.abspath('__file__')))

# .env 파일 로드
from dotenv import load_dotenv
load_dotenv()

print("✅ 환경 설정 완료")
print(f"📍 작업 디렉토리: {os.getcwd()}")
print(f"🔑 OpenAI API Key: {'설정됨' if os.getenv('OPENAI_API_KEY') else '❌ 없음'}")

✅ 환경 설정 완료
📍 작업 디렉토리: c:\SSAFY\third_project\AI
🔑 OpenAI API Key: 설정됨


In [33]:
# LangChain 체인 import
from app.chains.log_analysis_chain import log_analysis_chain
from app.chains.chatbot_chain import chatbot_chain

print("✅ LangChain 체인 로드 완료")
print(f"  - log_analysis_chain: {type(log_analysis_chain)}")
print(f"  - chatbot_chain: {type(chatbot_chain)}")

✅ LangChain 체인 로드 완료
  - log_analysis_chain: <class 'langchain_core.runnables.base.RunnableSequence'>
  - chatbot_chain: <class 'langchain_core.runnables.base.RunnableSequence'>


## 2. 로그 분석 체인 테스트

다양한 에러 로그에 대해 AI가 올바르게 분석하는지 확인합니다.

In [34]:
# 테스트 케이스 정의
test_logs = [
    {
        "name": "NullPointerException",
        "data": {
            "service_name": "user-service",
            "level": "ERROR",
            "message": "Failed to process user request",
            "method_name": "getUser",
            "class_name": "UserService",
            "timestamp": "2025-01-02T10:30:00Z",
            "duration": 150,
            "stack_trace": """java.lang.NullPointerException: Cannot invoke 'User.getName()' because 'user' is null
    at com.example.user.UserService.getUser(UserService.java:45)
    at com.example.user.UserController.getUserInfo(UserController.java:23)""",
            "additional_context": {"user_id": "12345"}
        }
    },
    {
        "name": "DatabaseConnectionException",
        "data": {
            "service_name": "order-service",
            "level": "ERROR",
            "message": "Database connection timeout",
            "method_name": "createOrder",
            "class_name": "OrderRepository",
            "timestamp": "2025-01-02T11:00:00Z",
            "duration": 30000,
            "stack_trace": """org.springframework.dao.DataAccessResourceFailureException: Unable to acquire JDBC Connection
    at org.springframework.jdbc.datasource.DataSourceUtils.getConnection(DataSourceUtils.java:82)""",
            "additional_context": {"pool_size": 10, "active_connections": 10}
        }
    },
    {
        "name": "정상 로그 (INFO)",
        "data": {
            "service_name": "payment-service",
            "level": "INFO",
            "message": "Payment processed successfully",
            "method_name": "processPayment",
            "class_name": "PaymentService",
            "timestamp": "2025-01-02T12:00:00Z",
            "duration": 250,
            "stack_trace": "N/A",
            "additional_context": {"amount": 50000, "method": "card"}
        }
    }
]

print(f"📋 총 {len(test_logs)}개 테스트 케이스 준비 완료")

📋 총 3개 테스트 케이스 준비 완료


In [35]:
# 로그 분석 실행
analysis_results = []

for i, test_case in enumerate(test_logs, 1):
    print(f"\n{'='*60}")
    print(f"🧪 테스트 {i}/{len(test_logs)}: {test_case['name']}")
    print(f"{'='*60}")
    
    start_time = time.time()
    
    try:
        # LLM 체인 실행
        result = log_analysis_chain.invoke(test_case['data'])
        
        elapsed_time = time.time() - start_time
        
        print(f"\n✅ 분석 완료 ({elapsed_time:.2f}초)")
        print(f"\n📊 분석 결과:")
        print(f"  - Summary: {result.summary}")
        print(f"  - Error Cause: {result.error_cause or 'N/A'}")
        print(f"  - Solution: {result.solution or 'N/A'}")
        print(f"  - Tags: {', '.join(result.tags)}")
        
        analysis_results.append({
            "test_name": test_case['name'],
            "status": "✅ 성공",
            "time": f"{elapsed_time:.2f}s",
            "summary": result.summary,
            "tags_count": len(result.tags)
        })
        
    except Exception as e:
        elapsed_time = time.time() - start_time
        print(f"\n❌ 실패: {e}")
        analysis_results.append({
            "test_name": test_case['name'],
            "status": "❌ 실패",
            "time": f"{elapsed_time:.2f}s",
            "error": str(e)
        })

print(f"\n\n{'='*60}")
print("📈 로그 분석 테스트 요약")
print(f"{'='*60}")


🧪 테스트 1/3: NullPointerException

❌ 실패: <lambda>() missing 1 required positional argument: 'inputs'

🧪 테스트 2/3: DatabaseConnectionException

❌ 실패: <lambda>() missing 1 required positional argument: 'inputs'

🧪 테스트 3/3: 정상 로그 (INFO)

❌ 실패: <lambda>() missing 1 required positional argument: 'inputs'


📈 로그 분석 테스트 요약


In [36]:
# 결과 요약 테이블
import pandas as pd

df_analysis = pd.DataFrame(analysis_results)
print("\n로그 분석 테스트 결과:")
display(df_analysis)

success_count = sum(1 for r in analysis_results if '✅' in r['status'])
print(f"\n성공률: {success_count}/{len(analysis_results)} ({success_count/len(analysis_results)*100:.1f}%)")


로그 분석 테스트 결과:


,test_name,status,time,error
0,NullPointerException,❌ 실패,0.00s,<lambda>() missing 1 required positional argum...
1,DatabaseConnectionException,❌ 실패,0.00s,<lambda>() missing 1 required positional argum...
2,정상 로그 (INFO),❌ 실패,0.00s,<lambda>() missing 1 required positional argum...



성공률: 0/3 (0.0%)


## 3. 챗봇 RAG 체인 테스트

다양한 질문에 대해 챗봇이 올바르게 응답하는지 확인합니다.

In [37]:
# 챗봇 테스트 케이스
chatbot_tests = [
    {
        "name": "한국어 - 에러 조회",
        "context_logs": """Log 1:
- ID: 12345
- Timestamp: 2025-01-02T10:30:00Z
- Service: user-service
- Level: ERROR
- Message: NullPointerException in UserService.getUser()
- Method: UserService.getUser
- Stack Trace: java.lang.NullPointerException: Cannot invoke 'User.getName()' because 'user' is null

Log 2:
- ID: 12346
- Timestamp: 2025-01-02T10:35:00Z
- Service: user-service
- Level: ERROR
- Message: NullPointerException in UserService.updateUser()
- Method: UserService.updateUser
- Stack Trace: java.lang.NullPointerException: Cannot invoke 'User.setName()' because 'user' is null""",
        "question": "최근에 어떤 에러가 발생했어?"
    },
    {
        "name": "영어 - 에러 조회",
        "context_logs": """Log 1:
- ID: 12347
- Timestamp: 2025-01-02T11:00:00Z
- Service: order-service
- Level: ERROR
- Message: Database connection timeout
- Method: OrderRepository.createOrder
- Stack Trace: org.springframework.dao.DataAccessResourceFailureException""",
        "question": "What errors occurred recently?"
    },
    {
        "name": "한국어 - 해결 방법 질문",
        "context_logs": """Log 1:
- ID: 12345
- Timestamp: 2025-01-02T10:30:00Z
- Service: user-service
- Level: ERROR
- Message: NullPointerException in UserService.getUser()
- Stack Trace: java.lang.NullPointerException: Cannot invoke 'User.getName()' because 'user' is null""",
        "question": "NullPointerException을 어떻게 해결하면 돼?"
    },
    {
        "name": "컨텍스트 없음",
        "context_logs": "No relevant logs found.",
        "question": "최근 에러 알려줘"
    }
]

print(f"📋 총 {len(chatbot_tests)}개 챗봇 테스트 케이스 준비 완료")

📋 총 4개 챗봇 테스트 케이스 준비 완료


In [38]:
# 챗봇 테스트 실행
chatbot_results = []

for i, test_case in enumerate(chatbot_tests, 1):
    print(f"\n{'='*60}")
    print(f"💬 테스트 {i}/{len(chatbot_tests)}: {test_case['name']}")
    print(f"{'='*60}")
    print(f"❓ 질문: {test_case['question']}")
    
    start_time = time.time()
    
    try:
        # 챗봇 체인 실행
        response = chatbot_chain.invoke({
            "context_logs": test_case['context_logs'],
            "question": test_case['question']
        })
        
        elapsed_time = time.time() - start_time
        answer = response.content
        
        print(f"\n✅ 응답 생성 완료 ({elapsed_time:.2f}초)")
        print(f"\n💡 답변:")
        print(f"  {answer}")
        
        chatbot_results.append({
            "test_name": test_case['name'],
            "question": test_case['question'],
            "status": "✅ 성공",
            "time": f"{elapsed_time:.2f}s",
            "answer_length": len(answer),
            "answer_preview": answer[:100] + "..." if len(answer) > 100 else answer
        })
        
    except Exception as e:
        elapsed_time = time.time() - start_time
        print(f"\n❌ 실패: {e}")
        chatbot_results.append({
            "test_name": test_case['name'],
            "question": test_case['question'],
            "status": "❌ 실패",
            "time": f"{elapsed_time:.2f}s",
            "error": str(e)
        })

print(f"\n\n{'='*60}")
print("📈 챗봇 테스트 요약")
print(f"{'='*60}")


💬 테스트 1/4: 한국어 - 에러 조회
❓ 질문: 최근에 어떤 에러가 발생했어?

❌ 실패: Error code: 401 - {'message': '[GMS 에러] Invalid or expired GMS key', 'statusCode': 401}

💬 테스트 2/4: 영어 - 에러 조회
❓ 질문: What errors occurred recently?

❌ 실패: Error code: 401 - {'message': '[GMS 에러] Invalid or expired GMS key', 'statusCode': 401}

💬 테스트 3/4: 한국어 - 해결 방법 질문
❓ 질문: NullPointerException을 어떻게 해결하면 돼?

❌ 실패: Error code: 401 - {'message': '[GMS 에러] Invalid or expired GMS key', 'statusCode': 401}

💬 테스트 4/4: 컨텍스트 없음
❓ 질문: 최근 에러 알려줘

❌ 실패: Error code: 401 - {'message': '[GMS 에러] Invalid or expired GMS key', 'statusCode': 401}


📈 챗봇 테스트 요약


In [39]:
# 챗봇 결과 요약
df_chatbot = pd.DataFrame(chatbot_results)
print("\n챗봇 테스트 결과:")
display(df_chatbot)

success_count = sum(1 for r in chatbot_results if '✅' in r['status'])
print(f"\n성공률: {success_count}/{len(chatbot_results)} ({success_count/len(chatbot_results)*100:.1f}%)")


챗봇 테스트 결과:


,test_name,question,status,time,error
0,한국어 - 에러 조회,최근에 어떤 에러가 발생했어?,❌ 실패,0.05s,Error code: 401 - {'message': '[GMS 에러] Invali...
1,영어 - 에러 조회,What errors occurred recently?,❌ 실패,0.02s,Error code: 401 - {'message': '[GMS 에러] Invali...
2,한국어 - 해결 방법 질문,NullPointerException을 어떻게 해결하면 돼?,❌ 실패,0.01s,Error code: 401 - {'message': '[GMS 에러] Invali...
3,컨텍스트 없음,최근 에러 알려줘,❌ 실패,0.02s,Error code: 401 - {'message': '[GMS 에러] Invali...



성공률: 0/4 (0.0%)


## 4. 전체 결과 요약

In [40]:
print("\n" + "="*60)
print("🎯 전체 테스트 결과 요약")
print("="*60)

total_tests = len(analysis_results) + len(chatbot_results)
total_success = sum(1 for r in analysis_results if '✅' in r['status']) + \
                sum(1 for r in chatbot_results if '✅' in r['status'])

print(f"\n📊 통계:")
print(f"  - 총 테스트: {total_tests}개")
print(f"  - 성공: {total_success}개")
print(f"  - 실패: {total_tests - total_success}개")
print(f"  - 성공률: {total_success/total_tests*100:.1f}%")

print(f"\n📈 카테고리별:")
analysis_success = sum(1 for r in analysis_results if '✅' in r['status'])
chatbot_success = sum(1 for r in chatbot_results if '✅' in r['status'])

print(f"  - 로그 분석: {analysis_success}/{len(analysis_results)} ({analysis_success/len(analysis_results)*100:.1f}%)")
print(f"  - 챗봇: {chatbot_success}/{len(chatbot_results)} ({chatbot_success/len(chatbot_results)*100:.1f}%)")

if total_success == total_tests:
    print("\n✅✅✅ 모든 테스트 통과! ✅✅✅")
else:
    print(f"\n⚠️ {total_tests - total_success}개 테스트 실패")


🎯 전체 테스트 결과 요약

📊 통계:
  - 총 테스트: 7개
  - 성공: 0개
  - 실패: 7개
  - 성공률: 0.0%

📈 카테고리별:
  - 로그 분석: 0/3 (0.0%)
  - 챗봇: 0/4 (0.0%)

⚠️ 7개 테스트 실패


## 5. 응답 시간 시각화 (선택)

In [41]:
import matplotlib.pyplot as plt

# 응답 시간 추출 (성공한 것만)
analysis_times = [float(r['time'].replace('s', '')) for r in analysis_results if '✅' in r['status']]
chatbot_times = [float(r['time'].replace('s', '')) for r in chatbot_results if '✅' in r['status']]

if analysis_times and chatbot_times:
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))
    
    # 로그 분석 응답 시간
    ax[0].bar(range(len(analysis_times)), analysis_times, color='skyblue')
    ax[0].set_title('Log Analysis Response Time')
    ax[0].set_xlabel('Test Case')
    ax[0].set_ylabel('Time (seconds)')
    ax[0].axhline(y=sum(analysis_times)/len(analysis_times), color='r', linestyle='--', label='Average')
    ax[0].legend()
    
    # 챗봇 응답 시간
    ax[1].bar(range(len(chatbot_times)), chatbot_times, color='lightgreen')
    ax[1].set_title('Chatbot Response Time')
    ax[1].set_xlabel('Test Case')
    ax[1].set_ylabel('Time (seconds)')
    ax[1].axhline(y=sum(chatbot_times)/len(chatbot_times), color='r', linestyle='--', label='Average')
    ax[1].legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n평균 응답 시간:")
    print(f"  - 로그 분석: {sum(analysis_times)/len(analysis_times):.2f}초")
    print(f"  - 챗봇: {sum(chatbot_times)/len(chatbot_times):.2f}초")
else:
    print("⚠️ 응답 시간 데이터 없음 (모든 테스트 실패)")

⚠️ 응답 시간 데이터 없음 (모든 테스트 실패)


## 6. 결론

**테스트 완료!**

이 노트북은 다음을 확인했습니다:
- ✅ LangChain 체인이 올바르게 로드되는지
- ✅ 다양한 로그에 대해 AI가 정확하게 분석하는지
- ✅ 챗봇이 한국어/영어 질문에 적절히 응답하는지
- ✅ LLM 응답 품질
- ✅ 응답 시간

**다음 단계**:
1. 실패한 테스트가 있다면 원인 분석
2. 전체 시스템 통합 테스트 (API 서버, OpenSearch 포함)
3. 프로덕션 배포 전 성능 테스트